In [198]:
# import necessary packages
import transformers
import pandas as pd
from datasets import Dataset, load_metric
import numpy as np
from torch.nn import CrossEntropyLoss
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer

### Prepare the Dataset


In [199]:
# load dataset as pandas Dataframe
df = pd.read_csv("Training Data.csv")

# check
df

,Email Text,Category
0,Thank you for your help.,Thank You
1,I appreciate your assistance.,Thank You
2,Grateful for your support.,Thank You
3,Thanks a lot!,Thank You
4,I'm going through a tough time. Can you help?,Other
5,I need assistance with a difficult situation.,Other
6,"Please, I'm in a crisis. Can you assist me?",Other
7,I'll think about it and get back to you.,Other
8,"Hello, how are you doing?",Other
9,I'm struggling right now. Can you help?,Other


In [200]:
# load the dataframe in a hugging face compatible format
dataset = Dataset.from_pandas(df)

# check the type
type(dataset)

datasets.arrow_dataset.Dataset

### Preprocessing the dataset


In [201]:
# encode the dataset labels as integers
dataset = dataset.class_encode_column('Category')

Casting to class labels: 100%|██████████| 53/53 [00:00<?, ? examples/s]


In [202]:
# view a sample of the dataset
dataset[2]

{'Email Text': 'Grateful for your support.', 'Category': 1}

Within Category, 1 is Thank You, 0 is Other


In [203]:
# verify the dataset features
dataset.features

{'Email Text': Value(dtype='string', id=None),
 'Category': ClassLabel(names=['Other', 'Thank You'], id=None)}

### Tokenization


In [204]:
# declare the checkpoint
checkpoint = "bert-base-uncased"

# call the tokenizer for training
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [205]:
# create a function for tokenizing the sample_mails
def tokenize_function(example):
    return tokenizer(example["Email Text"], truncation=True)

In [206]:
# tokenize the dataset with the map function
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Map: 100%|██████████| 53/53 [00:00<00:00, 2136.89 examples/s]


Dataset({
    features: ['Email Text', 'Category', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 53
})

In [207]:
# apply dynamic padding -- pad all the sample_mails to the length of the longest element when we batch elements together
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

To test this new toy, we'll slice our dataset that we would like to batch together. Here, we remove the columns idx and sample_mails as they won’t be needed and contain strings (and we can’t create tensors with strings) and have a look at the lengths of each entry in the batch:


In [208]:
# samples = tokenized_datasets[:]
# samples = {k: v for k, v in samples.items() if k not in ["idx", "Email Text"]}
# [len(x) for x in samples["input_ids"]]
fields_to_remove = ["Email Text"]

# Remove the specified fields
tokenized_datasets = tokenized_datasets.remove_columns(fields_to_remove)

No surprise, we get samples of varying length, from 7 to 15. Dynamic padding means the samples in this batch should all be padded to a length of 15, the maximum length inside the batch. Without dynamic padding, all of the samples would have to be padded to the maximum length in the whole dataset, or the maximum length the model can accept. Let’s double-check that our data_collator is dynamically padding the batch properly:


In [209]:
# we can convert the tokenized dataset back to text as follows
tokenizer.convert_ids_to_tokens(tokenized_datasets['input_ids'][-1])

['[CLS]', 'please', 'clarify', 'your', 'previous', 'message', '.', '[SEP]']

### Training

The first step before we can define our Trainer is to define a TrainingArguments class that will contain all the hyperparameters the Trainer will use for training and evaluation. The only argument you have to provide is a directory where the trained model will be saved, in our case we want to also modify the number of epochs for training, the checkpoints along the way are also saved in this directory. For all the rest, you can leave the defaults, which should work pretty well for a basic fine-tuning.


In [210]:
# define a metric to monitor during training
metric = load_metric("accuracy")

# create a function that helps compute the specified metric


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [211]:
# define the training arguments
training_args = TrainingArguments('training_args',
                                  num_train_epochs=20)

The second step is to define our model. We will use the AutoModelForSequenceClassification class, with two labels:


In [212]:


model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2)

KeyboardInterrupt: 

You will notice that you get a warning after instantiating this pretrained model. This is because BERT has not been pretrained on classifying pairs of sentences, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added instead. The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.


In [ ]:


# define trainer object
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)



You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyError: 'Category'

To fine-tune the model on our dataset, we just have to call the train() method of our Trainer:


In [ ]:
# train the model
trainer.train()

  0%|          | 0/140 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
# save the trained model together with the tokenizer in a directory
trainer.save_model('custom_model')

### Evaluation

For this task, we will evaluate the model on the training set, given that the dataset is extremely small and could not be split into train-test sets


In [ ]:
predictions = trainer.predict(tokenized_datasets)
print(predictions.predictions.shape, predictions.label_ids.shape, '\n')
print(predictions)

100%|██████████| 2/2 [00:01<00:00,  1.90it/s]

(11, 2) (11,) 

PredictionOutput(predictions=array([[-1.9196216,  1.3245827],
       [-1.949448 ,  1.3339628],
       [ 1.7652345, -1.1058381],
       [-2.0256405,  1.362749 ],
       [-2.0248377,  1.3859997],
       [ 1.6743466, -1.0574694],
       [ 1.7554849, -1.0992212],
       [-2.0404794,  1.3920761],
       [-1.9952593,  1.3621099],
       [ 1.7081995, -1.0845065],
       [-2.0348525,  1.3852504]], dtype=float32), label_ids=array([1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1], dtype=int64), metrics={'test_loss': 0.04296018183231354, 'test_accuracy': 1.0, 'test_runtime': 1.6477, 'test_samples_per_second': 6.676, 'test_steps_per_second': 1.214})


The output of the `predict()` method is another named tuple with three fields: predictions, `label_ids`, and `metrics`. The metrics field now contains the loss on the dataset passed, some time metrics (how long it took to predict, in total and on average), and the accuracy of training

As we can see, predictions is a two-dimensional array with shape 11 x 2 (11 being the number of elements in the dataset we used). Those are the logits for each element of the dataset we passed to `predict()`. To transform them into predictions that we can compare to our labels, we need to take the index with the maximum value on the second axis:


In [ ]:
preds = np.argmax(predictions.predictions, axis=-1)

We can now compare those preds to the labels. To build our `compute_metric()` function, we will rely on the metrics from the 🤗 Datasets library. We can load the metrics associated with the MRPC dataset as easily as we loaded the dataset, this time with the `load_metric()` function. The object returned has a `compute()` method we can use to do the metric calculation. Wrapping everything together, we get our `compute_metrics_mrpc()` function:


In [ ]:
def compute_metrics_mrpc(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
compute_metrics_mrpc(predictions)

{'accuracy': 1.0, 'f1': 1.0}

In [ ]:
predictions.predictions

array([[-1.9196216,  1.3245827],
       [-1.949448 ,  1.3339628],
       [ 1.7652345, -1.1058381],
       [-2.0256405,  1.362749 ],
       [-2.0248377,  1.3859997],
       [ 1.6743466, -1.0574694],
       [ 1.7554849, -1.0992212],
       [-2.0404794,  1.3920761],
       [-1.9952593,  1.3621099],
       [ 1.7081995, -1.0845065],
       [-2.0348525,  1.3852504]], dtype=float32)

In [ ]:
predictions.label_ids

array([1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1], dtype=int64)

From the above, we see that the model has a perfect prediction on the data it was trained on. This is highly flawed and can be ascribed to overfitting, but since we have no test set to evaluate on given the size of the sample data, we can assume that for the model to overfit at 20 epochs, it actually did well in learning the training dataset.

### Inference


In [ ]:
# get the directory where the model was saved to
inf_model = AutoModelForSequenceClassification.from_pretrained('custom_model/')

I had to change the path of the custom model to 'custom_model/'


In [ ]:
# load the tokenizer by pointing to the same directory as the pretrained model
inf_tokenizer = AutoTokenizer.from_pretrained('custom_model/')

In [ ]:
# generate sequence for inference
sequences = ['I want to know if I should send your email',
             'I sent your email a long time ago']

In [ ]:
# create a pipeline for inference
from transformers import pipeline
classifier = pipeline(task='text-classification',
                      model=inf_model, tokenizer=inf_tokenizer)

In [ ]:
classifier(sequences)

[{'label': 'LABEL_1', 'score': 0.9348370432853699},
 {'label': 'LABEL_0', 'score': 0.7933637499809265}]

From the above output, we can confidently say the model is performing well on inference
